<a href="https://colab.research.google.com/github/icanello01/krateras/blob/main/krateras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Krateras 🚀✨🔒: O Especialista Robótico de Denúncia de Buracos (v3.2 - Maximum Secure Colab Edition)

Olá! Krateras v3.2 entrando em órbita! A segurança dos seus dados é minha prioridade máxima.
Agora, eu vou tentar obter AMBAS as suas chaves de API (Gemini e Geocoding) dos Segredos
do Google Colab, mantendo-as TOTALMENTE fora do código fonte visível!

Fui criado com o que há de mais avançado em Programação, IA, Design Inteligente,
Matemática (ainda essencial!) e Lógica Inabalável. Com acesso seguro às APIs, sou imparável.

Vamos juntos consertar essas ruas! Iniciando varredura de chaves...
"""

import sys
import subprocess
import json
import os
import time
from typing import Dict, Any, Optional
import re
import textwrap

# --- Tentar importar google.colab.userdata para acessar Segredos ---
# Isso precisa ser feito no início para verificarmos a disponibilidade do ambiente.
try:
    from google.colab import userdata
    COLAB_SECRETS_AVAILABLE = True
    print("✅ Módulo google.colab.userdata disponível. Ambiente Colab detectado. Tentarei usar Segredos.")
except ImportError:
    COLAB_SECRETS_AVAILABLE = False
    print("⚠️ Módulo google.colab.userdata não disponível. Não estou rodando no Colab ou acesso aos Segredos está restrito.")
    print("Solicitarei as chaves de API manualmente via input.")


# --- ⚙️ Configuração Inicial: Dependências ---
# Garante que temos as ferramentas certas instaladas.

required_libraries: list[str] = ['requests', 'google-generativeai'] # Requests para ViaCEP e Geocoding API, Gemini para IA

def install_dependencies(libraries: list[str]):
    """Verifica e instala bibliotecas necessárias no ambiente Colab."""
    print("✨ Krateras v3.2 Inicializando seus sistemas...")
    print("🔧 Realizando varredura de componentes essenciais e instalando ferramentas...")
    for lib in libraries:
        try:
            __import__(lib)
            # print(f"'{lib}' detectado. Componente online.") # Comentado para output mais limpo
        except ImportError:
            print(f"🛰️ Ferramenta '{lib}' não encontrada. Baixando e instalando do meu repositório central...")
            try:
                # Usamos Popen para evitar travar e capturar output de instalação
                process = subprocess.Popen([sys.executable, "-m", "pip", "install", lib], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                stdout, stderr = process.communicate()
                if process.returncode != 0:
                     raise subprocess.CalledProcessError(process.returncode, process.args, output=stdout, stderr=stderr)
                print(f"'{lib}' instalado com sucesso. Montagem completa!")
            except subprocess.CalledProcessError as e:
                print(f"❌ ERRO CRÍTICO no Hardware: Falha na instalação do componente '{lib}'.")
                print("Meu sistema está comprometido sem esta ferramenta. Tente instalar manualmente no Colab:")
                print(f"!pip install {lib}")
                print(f"Detalhes técnicos do erro:\n{e.stderr.decode()}")
                sys.exit(1)

install_dependencies(required_libraries)

# Importa as bibliotecas após garantir que estão instaladas
import requests
import google.generativeai as genai

# --- 🔑 Gerenciamento de Chaves Secretas (APIs) ---
# Prioriza Segredos do Colab para AMBAS as chaves.

def get_api_keys() -> tuple[Optional[str], Optional[str]]:
    """
    Tenta obter as chaves de API do Google Gemini e Google Maps Geocoding de Segredos do Colab.
    Solicita manualmente se não encontradas ou se não está no ambiente Colab.
    """
    print("\n--- 🔑 Inserindo Chaves de Acesso Seguras (Energia Secreta para os Robôs!) ---")
    print("Vou tentar carregar suas chaves dos Segredos do Colab para máxima segurança.")
    print("Certifique-se de ter os segredos 'GOOGLE_API_KEY' (para Gemini) e 'geocoding_api_key' (para Geocoding) configurados.")


    gemini_key: Optional[str] = None
    geocoding_key: Optional[str] = None

    # --- Tentar obter Chave Gemini de Segredos ---
    print("\n1. Chave Google AI Studio/Vertex AI (Gemini):")
    if COLAB_SECRETS_AVAILABLE:
        print("💾 Tentando obter a chave 'GOOGLE_API_KEY' dos Segredos do Colab...")
        try:
            gemini_key = userdata.get('GOOGLE_API_KEY')
            if gemini_key:
                print("✅ Chave 'GOOGLE_API_KEY' encontrada nos Segredos do Colab!")
            else:
                print("⚠️ Segredo 'GOOGLE_API_KEY' não encontrado nos Segredos do Colab.")
        except Exception as e:
            print(f"❌ Erro ao tentar acessar Segredo 'GOOGLE_API_KEY': {e}.")
            gemini_key = None # Garante que a chave seja None em caso de erro
    else:
        print("ℹ️ Segredos do Colab não disponíveis. Pular verificação de Segredo para Gemini.")


    # --- Tentar obter Chave Geocoding de Segredos ---
    print("\n2. Chave Google Cloud Platform (Geocoding API):")
    if COLAB_SECRETS_AVAILABLE:
        print("💾 Tentando obter a chave 'geocoding_api_key' dos Segredos do Colab...")
        try:
            geocoding_key = userdata.get('geocoding_api_key')
            if geocoding_key:
                print("✅ Chave 'geocoding_api_key' encontrada nos Segredos do Colab!")
            else:
                 print("⚠️ Segredo 'geocoding_api_key' não encontrado nos Segredos do Colab.")
        except Exception as e:
            print(f"❌ Erro ao tentar acessar Segredo 'geocoding_api_key': {e}.")
            geocoding_key = None # Garante que a chave seja None em caso de erro
    else:
        print("ℹ️ Segredos do Colab não disponíveis. Pular verificação de Segredo para Geocoding.")


    # --- Fallback para Input Manual se as chaves não foram encontradas nos Segredos ---

    if not gemini_key:
        print("\n➡️ Por favor, insira sua Chave de API do Google Gemini (se não configurou o segredo ou para usar outra chave).")
        print("   (Ou apenas Enter para pular - Análise IA será desabilitada).")
        gemini_key = input("Sua Chave de API do Google Gemini: ").strip()
        if not gemini_key:
            print("⚠️ Chave Gemini não fornecida. Análise de IA do Gemini desabilitada.")

    if not geocoding_key:
        print("\n➡️ Por favor, insira sua Chave de API do Google Maps Geocoding (se não configurou o segredo ou para usar outra chave).")
        print("   (Ou apenas Enter se não tiver/não quiser usar - Geocodificação automática desabilitada).")
        print("   ❗ Lembre-se que a API Geocoding PODE gerar custos. Ative-a no Google Cloud.")
        geocoding_key = input("Sua Chave de API do Google Maps Geocoding: ").strip()
        if not geocoding_key:
            print("⚠️ Chave de API de Geocoding não fornecida. Geocodificação automática desabilitada.")


    return gemini_key, geocoding_key

def init_gemini(api_key: str) -> Optional[genai.GenerativeModel]:
    """Inicializa o modelo Google Gemini."""
    if not api_key:
        print("\n⚠️ AVISO: Chave de API do Gemini não fornecida. As funcionalidades de IA do Gemini estão desabilitadas.")
        return None
    try:
        genai.configure(api_key=api_key)
        # Buscar modelos que suportam geração de conteúdo de texto
        available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

        if not available_models:
             print("\n❌ ERRO na Fábrica de Modelos: Nenhum modelo de texto Gemini compatível encontrado na sua conta.")
             return None

        preferred_models: list[str] = ['gemini-1.5-flash-latest', 'gemini-1.0-pro', 'gemini-pro']
        model_name: Optional[str] = None
        for pref_model in preferred_models:
            if f'models/{pref_model}' in available_models:
                model_name = pref_model
                break
            if pref_model in available_models: # Verifica sem o prefixo 'models/'
                 model_name = pref_model
                 break

        if not model_name:
            model_name = available_models[0].replace('models/', '')
            print(f"⚠️ AVISO: Modelos Gemini preferenciais não encontrados. Usando fallback: '{model_name}'. Pode ter menos funcionalidades.")

        model = genai.GenerativeModel(model_name)
        # Opcional: testar uma chamada básica para verificar se a API está funcionando
        # model.generate_content("Ping")
        print(f"✅ Conexão com Google Gemini estabelecida usando modelo '{model_name}'. A IA está online e pensativa!")
        return model
    except Exception as e:
        print(f"\n❌ ERRO no Painel de Controle Gemini: Falha na inicialização do Google Gemini. Verifique sua chave e status do serviço.")
        print(f"Detalhes técnicos do erro: {e}")
        return None

def geocodificar_endereco(rua: str, numero: str, cidade: str, estado: str, api_key: str) -> Dict[str, Any]:
    """Tenta obter coordenadas geográficas e link Google Maps via Google Maps Geocoding API."""
    if not api_key:
        return {"erro": "Chave de API de Geocodificação não fornecida."}
    # Adicionado validação básica para garantir que os campos essenciais não estejam vazios
    if not rua or not numero or not cidade or not estado:
         return {"erro": "Dados de endereço insuficientes (requer rua, número, cidade, estado) para geocodificar."}

    address = f"{rua}, {numero}, {cidade}, {estado}"
    # Use requests.utils.quote para garantir que o endereço seja formatado corretamente na URL
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={requests.utils.quote(address)}&key={api_key}"

    print(f"⏳ Tentando localizar '{address}' no mapa global via Geocoding API...")
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()

        if data['status'] != 'OK':
            status = data.get('status', 'STATUS DESCONHECIDO')
            error_msg = data.get('error_message', 'Sem mensagem adicional.')
            return {"erro": f"Geocodificação falhou. Status: {status}. Mensagem: {error_msg}"}
        if not data['results']:
             return {"erro": "Geocodificação falhou. Nenhum local exato encontrado para o endereço fornecido."}

        # Pegar o primeiro resultado
        location = data['results'][0]['geometry']['location']
        lat = location['lat']
        lng = location['lng']
        formatted_address = data['results'][0].get('formatted_address', address)

        return {
            "latitude": lat,
            "longitude": lng,
            "endereco_formatado_api": formatted_address, # Nome mais claro
            "google_maps_link_gerado": f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"
        }
    except requests.exceptions.Timeout:
         return {"erro": f"Tempo limite excedido ({15}s) ao tentar geocodificar: {address}"}
    except requests.exceptions.RequestException as e:
        return {"erro": f"Erro na comunicação com a API de Geocodificação: {address}. Detalhes: {e}"}
    except Exception as e:
        return {"erro": f"Ocorreu um erro inesperado durante a geocodificação: {address}. Detalhes: {e}"}


# --- 🤖 Funções de Coleta de Dados Inteligente ---
# Coletamos as informações, com validações e fluxo aprimorados.

def coletar_dados_denunciante() -> Dict[str, Any]:
    """Coleta informações do usuário denunciante com um toque amigável."""
    print("\n--- 👤 Dados do Herói/Heroína da Vez! ---")
    nome = input("Seu nome completo: ").strip()
    while not nome:
        print("❗ Nome não pode ser um mistério. Quem é você na fila do pão da cidadania?")
        nome = input("Seu nome completo: ").strip()

    idade_str = input("Sua idade (aproximada, se preferir, sem pressão 😉): ").strip()
    while not idade_str.isdigit() or int(idade_str) <= 0 or int(idade_str) > 120:
        print("❗ Idade inválida. Não se preocupe, robôs não julgam (ainda)!")
        idade_str = input("Sua idade: ").strip()
    idade = int(idade_str)

    # Este é a CIDADE DE RESIDÊNCIA DO DENUNCIANTE
    cidade = input("Em qual cidade você reside?: ").strip()
    while not cidade:
        print("❗ Cidade de residência não pode ficar vazia. Precisamos saber sua base de operações!")
        cidade = input("Em qual cidade você reside?: ").strip()

    print(f"Olá, {nome}! Sua contribuição é super valiosa! 💪")
    return {
        "nome": nome,
        "idade": idade,
        "cidade_residencia": cidade # Nome da chave mais explícito
    }

def buscar_cep(cep: str) -> Dict[str, Any]:
    """Consulta a API ViaCEP para obter dados de endereço com tratamento de erros."""
    cep_limpo = cep.replace("-", "").replace(".", "").strip()
    if len(cep_limpo) != 8 or not cep_limpo.isdigit():
        return {"erro": "Formato de CEP inválido. Precisa de 8 dígitos, amigão!"}

    url = f"https://viacep.com.br/ws/{cep_limpo}/json/"
    print(f"⏳ Interrogando o ViaCEP para o CEP '{cep_limpo}'...")
    try:
        response = requests.get(url, timeout=15) # Aumenta o timeout um pouco
        response.raise_for_status() # Lança exceção para status de erro (4xx ou 5xx)
        data = response.json()
        if 'erro' in data and data['erro'] is True:
            return {"erro": f"CEP '{cep_limpo}' não encontrado no ViaCEP. Ele se escondeu! 🧐"}
        # Verificar se os campos essenciais estão presentes
        if not data.get('logradouro') or not data.get('localidade') or not data.get('uf'):
             return {"erro": f"CEP '{cep_limpo}' encontrado, mas os dados de endereço estão incompletos. O ViaCEP só contou parte da história!"}
        return data
    except requests.exceptions.Timeout:
         return {"erro": f"Tempo limite excedido ({15}s) ao buscar o CEP '{cep_limpo}'. O ViaCEP não responde! 😴"}
    except requests.exceptions.RequestException as e:
        return {"erro": f"Erro na comunicação com o ViaCEP para o CEP '{cep_limpo}': {e}. Problemas na linha!"}
    except Exception as e:
         return {"erro": f"Ocorreu um erro inesperado ao buscar o CEP '{cep_limpo}': {e}. Isso não estava nos meus manuais!"}


def coletar_dados_buraco(geocoding_api_key: Optional[str]) -> Dict[str, Any]:
    """Coleta informações detalhadas sobre o buraco, com fluxo aprimorado, geocodificação condicional e IA."""
    print("\n--- 🚧 Detalhes do Buraco (Nosso Alvo!) ---")

    endereco: Dict[str, str] = {} # Dados básicos de rua, bairro, cidade, estado do buraco
    cep_informado: str = "" # Armazena o CEP se o usuário o forneceu
    rua_coletada_com_sucesso: bool = False

    while not rua_coletada_com_sucesso:
        opcao_localizacao = input("Como identificar a rua do buraco? (1 - Digitar nome manualmente | 2 - Buscar por CEP): ").strip()

        if opcao_localizacao == '1':
            endereco['rua'] = input("Nome completo da rua: ").strip()
            while not endereco.get('rua'):
                 print("❗ O nome da rua é o mínimo que precisamos! Onde está o buraco?")
                 endereco['rua'] = input("Nome completo da rua: ").strip()
            # Este é a CIDADE DO BURACO (quando inserida manualmente)
            endereco['cidade_buraco'] = input("Cidade onde está o buraco: ").strip()
            while not endereco.get('cidade_buraco'):
                 print("❗ Precisamos da cidade onde está o buraco.")
                 endereco['cidade_buraco'] = input("Cidade onde está o buraco: ").strip()
            endereco['estado_buraco'] = input("Estado (UF) onde está o buraco: ").strip()
            while not endereco.get('estado_buraco'):
                 print("❗ E o estado do buraco também é importante!")
                 endereco['estado_buraco'] = input("Estado (UF) onde está o buraco: ").strip()
            endereco['bairro'] = input("Bairro onde está o buraco (opcional): ").strip()
            rua_coletada_com_sucesso = True
        elif opcao_localizacao == '2':
            print("--- Busca por CEP ---")
            print("Digite o CEP do local do buraco (ou 'm' para voltar e digitar a rua manualmente).")
            while True: # Loop interno para tentativas de CEP
                cep_input = input("Digite o CEP ou 'm': ").strip().lower()

                if cep_input == 'm':
                    print("Ok. Alternando para entrada manual da rua.")
                    break # Sai do loop interno de CEP
                if not cep_input:
                     print("❗ Entrada não pode ser vazia. Forneça um CEP válido ou 'm'!")
                     continue # Pede CEP ou 'm' novamente

                dados_cep = buscar_cep(cep_input)

                if 'erro' in dados_cep:
                    print(f"❌ Falha na busca por CEP: {dados_cep['erro']}")
                    continue # Loop interno continua
                else: # CEP encontrado com sucesso
                    cep_informado = cep_input
                    print("\n--- ✅ Endereço Encontrado (ViaCEP) ---")
                    print(f"Rua: {dados_cep.get('logradouro', 'Não informado')}")
                    print(f"Bairro: {dados_cep.get('bairro', 'Não informado')}")
                    print(f"Cidade: {dados_cep.get('localidade', 'Não informado')}") # localidade do CEP
                    print(f"Estado: {dados_cep.get('uf', 'Não informado')}") # uf do CEP
                    print(f"CEP: {cep_informado}")
                    print("--------------------------------------")

                    confirmacao = input("As informações acima parecem corretas? (s/n, padrão 's'): ").strip().lower()
                    if confirmacao == 's' or confirmacao == '':
                        endereco['rua'] = dados_cep.get('logradouro', '')
                        endereco['bairro'] = dados_cep.get('bairro', '')
                        endereco['cidade_buraco'] = dados_cep.get('localidade', '') # Cidade do Buraco via CEP
                        endereco['estado_buraco'] = dados_cep.get('uf', '')     # Estado do Buraco via CEP
                        print("✅ Endereço confirmado. Precisão robótica aprovada!")
                    else:
                        print("❗ Endereço do CEP não confirmado. Vamos corrigir manualmente.")
                        endereco['rua'] = input("Nome da rua CORRETA: ").strip()
                        while not endereco.get('rua'):
                             print("❗ Nome da rua é essencial. Qual o nome CORRETO?")
                             endereco['rua'] = input("Nome da rua CORRETA: ").strip()
                        endereco['bairro'] = input("Bairro CORRETO (opcional): ").strip()
                         # Usa cidade/estado do CEP como padrão se não informado manualmente, para facilitar
                        endereco['cidade_buraco'] = input(f"Cidade CORRETA (Padrão: {dados_cep.get('localidade', 'Não informado')}): ").strip() or dados_cep.get('localidade', '')
                        while not endereco.get('cidade_buraco'): # Garante que a cidade não fique vazia
                            print("❗ Cidade do buraco não pode ser vazia. Digite a cidade CORRETA.")
                            endereco['cidade_buraco'] = input("Cidade CORRETA: ").strip()

                        endereco['estado_buraco'] = input(f"Estado CORRETO (UF) (Padrão: {dados_cep.get('uf', 'Não informado')}): ").strip() or dados_cep.get('uf', '')
                        while not endereco.get('estado_buraco'): # Garante que o estado não fique vazio
                             print("❗ Estado do buraco é obrigatório. Digite o Estado CORRETO.")
                             endereco['estado_buraco'] = input("Estado CORRETO (UF): ").strip()

                        print("✅ Endereço corrigido manualmente. Flexibilidade é meu forte!")

                    rua_coletada_com_sucesso = True # Sai do loop principal de rua
                    break # Sai do loop interno de CEP
            # Se o loop interno terminou por 'm', o loop externo continua.
        else:
            print("❗ Opção inválida. Escolha 1 ou 2. Minha lógica só entende isso por enquanto!")

    # --- Coletar Número Próximo e Lado da Rua ---
    numero_proximo = input("Número do imóvel mais próximo ou ponto de referência (Ex: 'Em frente ao 123', 'Esquina c/ Rua X'. ESSENCIAL para precisão!): ").strip()
    while not numero_proximo:
        print("❗ O número próximo/referência é VITAL para localizar o buraco precisamente. Por favor, informe!")
        numero_proximo = input("Número do imóvel mais próximo ou ponto de referência: ").strip()

    lado_rua = input("Lado da rua onde está o buraco (Ex: 'lado par', 'lado ímpar', 'lado direito', 'lado esquerdo'. Ajuda a identificar o ponto exato): ").strip()
    while not lado_rua:
         print("❗ Saber o lado da rua afina a busca no mapa. Qual é o lado?")
         lado_rua = input("Lado da rua: ").strip()

    # --- 📍 Coleta de Localização Exata (Geocodificação com Link Google Maps ou Manual) ---
    print("\n--- 📍 ONDE EXATAMENTE ESTÁ O BURACO? (Missão de Precisão!) ---")
    print("Agora, a localização exata para que o time de reparo vá DIRETO ao ponto!")
    print("Lembre-se: Não acesso seu GPS (estou na nuvem!).")

    localizacao_exata_processada: Dict[str, Any] = {"tipo": "Não informada"} # Dicionário para armazenar a localização final processada
    tentou_geocodificar: bool = False
    geocodificacao_sucesso: bool = False
    motivo_falha_geo: str = ""

    # Condição para tentar geocodificar: Tem chave de API E tem Rua, Número, Cidade do Buraco, Estado do Buraco
    tem_dados_para_geo = endereco.get('rua') and numero_proximo and endereco.get('cidade_buraco') and endereco.get('estado_buraco')

    if geocoding_api_key and tem_dados_para_geo:
        print("\n✅ Chave de Geocodificação e dados básicos de endereço completos encontrados.")
        print("⏳ Tentando gerar o link do Google Maps automaticamente a partir do endereço...")
        tentou_geocodificar = True
        geo_resultado = geocodificar_endereco(
            endereco['rua'],
            numero_proximo,
            endereco['cidade_buraco'],
            endereco['estado_buraco'],
            geocoding_api_key
        )

        if 'erro' not in geo_resultado:
            geocodificacao_sucesso = True
            localizacao_exata_processada = {
                "tipo": "Geocodificada (API)", # Indica que veio da API
                "latitude": geo_resultado['latitude'],
                "longitude": geo_resultado['longitude'],
                "endereco_formatado_api": geo_resultado.get('endereco_formatado_api', ''),
                "google_maps_link_gerado": geo_resultado['google_maps_link_gerado']
            }
            print("\n--- ✅ Localização Obtida (via Geocodificação Automática) ---")
            print(f"Endereço (API): {localizacao_exata_processada.get('endereco_formatado_api', 'Não disponível')}")
            print(f"Coordenadas: {localizacao_exata_processada.get('latitude')}, {localizacao_exata_processada.get('longitude')}")
            print(f"Link Google Maps: {localizacao_exata_processada.get('google_maps_link_gerado', 'Não disponível')}")
            print("---------------------------------------------------------")
            print("Esta é a localização que a API encontrou para o endereço. Se não for EXATA, por favor, forneça-a manualmente na próxima etapa!")

        else: # Erro na geocodificação
            print(f"❌ Falha na Geocodificação automática: {geo_resultado['erro']}")
            motivo_falha_geo = f"Erro da API de Geocodificação: {geo_resultado.get('erro', 'Motivo desconhecido')}"

    elif geocoding_api_key and not tem_dados_para_geo:
         print("⚠️ AVISO: Chave de Geocodificação fornecida, mas dados de endereço insuficientes (precisa de Rua, Número Próximo, Cidade, Estado).")
         print("A Geocodificação automática NÃO será tentada.")
         motivo_falha_geo = "Dados insuficientes para Geocodificação (requer Rua, Número, Cidade, Estado)."

    elif not geocoding_api_key:
         print("⚠️ AVISO: Chave de API de Geocodificação NÃO fornecida.")
         print("A Geocodificação automática NÃO será tentada.")
         motivo_falha_geo = "Chave de API de Geocodificação não fornecida."


    # Se a geocodificação NÃO FOI bem-sucedida OU NÃO FOI TENTADA, pedimos entrada manual
    # Usamos 'geocodificacao_sucesso' aqui. Se tentou mas falhou ou não tentou, pedimos manual.
    if not geocodificacao_sucesso:
        print("\n--- ✍️ Fornecer Localização Exata Manualmente (Alternativa ou Correção) ---")
        if motivo_falha_geo:
             print(f"(Motivo para entrada manual: {motivo_falha_geo})")

        print("➡️ Por favor, forneça a localização EXATA do buraco de forma manual:")
        print("   A MELHOR forma é COPIAR AS COORDENADAS (Lat,Long) ou um LINK do Google Maps que as contenha.")
        print("   Sugestão: Abra o Google Maps, encontre o buraco, TOQUE/CLIQUE E SEGURE NO LOCAL PREOCUPANTE. As coordenadas ou um link aparecerão.")
        print("   Alternativamente, uma DESCRIÇÃO MUITO DETALHADA do local EXATO no mapa.")

        localizacao_manual_input = input("Insira COORDENADAS (Lat,Long), LINK do Maps com Coordenadas, OU DESCRIÇÃO DETALHADA: ").strip()

        while not localizacao_manual_input:
             print("❗ A localização exata é a bússola para o reparo! Por favor, forneça COORDENADAS, LINK ou DESCRIÇÃO.")
             localizacao_manual_input = input("Insira COORDENADAS (Lat,Long), LINK do Maps com Coordenadas, OU DESCRIÇÃO DETALHADA: ").strip()

        # Tentar extrair coordenadas do input manual (reusando regex)
        lat: Optional[float] = None
        lon: Optional[float] = None
        tipo_manual_processado = "Descrição Manual Detalhada"

        # Regex para tentar achar coordenadas em diferentes formatos (Lat,Long ou em links comuns)
        # Tenta cobrir "lat,long", "@lat,long" em links
        match_coords = re.search(r'(-?\d+\.?\d*)[,\s/]+(-?\d+\.?\d*)', localizacao_manual_input)
        if match_coords:
            try:
                teste_lat = float(match_coords.group(1))
                teste_lon = float(match_coords.group(2))
                # Validação básica de faixa de coordenadas
                if -90 <= teste_lat <= 90 and -180 <= teste_lon <= 180:
                    lat = teste_lat
                    lon = teste_lon
                    tipo_manual_processado = "Coordenadas Fornecidas/Extraídas Manualmente"
                    print("✅ Coordenadas válidas detectadas no input manual! Navegação calibrada.")
                else:
                    print("⚠️ Parece um formato de coordenadas, mas fora da faixa esperada. Tratando como descrição. Robô confuso! 😅")
            except ValueError:
                print("ℹ️ Entrada manual não parece ser coordenadas válidas. Tratando como descrição detalhada.")
            except Exception as e:
                 print(f"ℹ️ Ocorreu um erro ao tentar processar as coordenadas/link no input manual: {e}. Tratando como descrição.")
        elif localizacao_manual_input.startswith("http"):
             print("ℹ️ Entrada manual é um link. Tentando extrair coordenadas (sujeito a formato do link)...")
             # Regex mais focado em links Google Maps que EM GERAL contém @lat,long
             match_maps_link = re.search(r'/@(-?\d+\.?\d*),(-?\d+\.?\d*)', localizacao_manual_input)
             if match_maps_link:
                 try:
                     teste_lat = float(match_maps_link.group(1))
                     teste_lon = float(match_maps_link.group(2))
                      # Validação básica de faixa
                     if -90 <= teste_lat <= 90 and -180 <= teste_lon <= 180:
                         lat = teste_lat
                         lon = teste_lon
                         tipo_manual_processado = "Coordenadas Extraídas de Link (Manual)"
                         print("✅ Coordenadas extraídas de link do Maps no input manual!")
                     else:
                         print("⚠️ Coordenadas extraídas do link no input manual fora da faixa esperada. Tratando como descrição.")
                 except ValueError:
                    print("ℹ️ Valores no link não parecem coordenadas válidas. Tratando como descrição.")
                 except Exception as e:
                      print(f"ℹ️ Ocorreu um erro ao tentar processar o link no input manual: {e}. Tratando como descrição.")
             else:
                  print("ℹ️ Não foi possível extrair coordenadas reconhecíveis do link fornecido manualmente.")
        else:
             print("ℹ️ Entrada manual não detectada como coordenadas ou link. Tratando como descrição detalhada.")


        # Armazenar o resultado do input manual no dicionário de localização processada
        if lat is not None and lon is not None:
            localizacao_exata_processada = {
                 "tipo": tipo_manual_processado,
                 "input_original": localizacao_manual_input,
                 "latitude": lat,
                 "longitude": lon,
                 "google_maps_link_gerado": f"https://www.google.com/maps/search/?api=1&query={lat},{lon}" # Gera link do Maps
            }
        else: # Não conseguiu extrair Lat/Long, armazena como descrição
            localizacao_exata_processada = {
                 "tipo": "Descrição Manual Detalhada",
                 "input_original": localizacao_manual_input,
                 "descricao_manual": localizacao_manual_input # A descrição é o input original
            }

        # Se houve uma tentativa de geocodificação automática que falhou, registra o motivo na entrada manual processada
        if tentou_geocodificar and not geocodificacao_sucesso:
             localizacao_exata_processada['motivo_falha_geocodificacao_anterior'] = motivo_falha_geo
        elif not geocoding_api_key:
             localizacao_exata_processada['motivo_falha_geocodificacao_anterior'] = "Chave de API de Geocodificação não fornecida."
        elif not tem_dados_para_geo and geocoding_api_key: # Tinha chave mas não dados suficientes
             localizacao_exata_processada['motivo_falha_geocodificacao_anterior'] = "Dados insuficientes para Geocodificação (Rua, Número, Cidade, Estado)."


    # --- 🧠 Entrada Principal para Análise de IA ---
    print("\n--- ✍️ Descreva o Buraco em Detalhes (Hora da IA Brilhar!) ---")
    print("Esta descrição alimenta o cérebro da IA. Quanto mais rico em detalhes, melhor a análise!")
    print("Fale sobre tamanho, profundidade, perigos, se tem água, se é antigo/novo, etc.")
    print("Pense nisso como um áudio para um amigo explicando o problema na rua. 😉")

    descricao_detalhada = input("Sua descrição detalhada do buraco: ").strip()
    while not descricao_detalhada:
         print("❗ A descrição é a matéria-prima da IA! Não deixe vazio, por favor.")
         descricao_detalhada = input("Sua descrição detalhada do buraco: ").strip()


    # --- Nota sobre a limitação de geolocalização direta ---
    print("\n--- 💡 Aviso Robótico Amigável ---")
    print("Lembre-se: Eu, Krateras, moro nas nuvens do Google Colab. Não tenho acesso ao seu GPS local!")
    print("A precisão da localização desta denúncia depende das informações (Geocodificação ou Coordenadas/Descrição Manual) que você me deu agora.")
    print("Agradeço sua cooperação geográfica! 🙏")
    print("------------------------------------------\n")


    return {
        "endereco": endereco, # Contém rua, bairro, cidade_buraco, estado_buraco
        "cep_informado": cep_informado, # Vazio se a rua foi digitada manualmente
        "numero_proximo": numero_proximo,
        "lado_rua": lado_rua,
        "localizacao_exata_processada": localizacao_exata_processada, # Dicionário detalhado da localização PROCESSADA
        "descricao_detalhada": descricao_detalhada
    }

# --- 🧠 Funções de Análise Avançada de IA (Google Gemini) ---
# Aqui o Google Gemini usa seus circuitos avançados para entender o buraco.

def analisar_descricao_gemini(descricao: str, model: genai.GenerativeModel) -> Dict[str, Any]:
    """Utiliza o Gemini para analisar a descrição detalhada do buraco e extrair insights estruturados."""
    if not model:
        return {"insights": "🤖 Análise de descrição via IA indisponível (Motor Gemini offline)."}
    if not descricao or descricao.strip() == "":
         return {"insights": "🔍 Sem descrição fornecida para análise de IA."}

    print("\n🧠 Executando análise profunda da descrição do buraco com IA Gemini...")
    try:
        prompt = f"""
        Analise a seguinte descrição DETALHADA de um buraco em uma rua. Seu objetivo é extrair informações objetivas e insights CRUCIAIS para um sistema de denúncias de reparo público.
        Formate a saída como um texto claro, usando marcadores (-) ou títulos para cada categoria.
        Se uma categoria NÃO PUDER ser claramente mencionada ou inferida COM ALTA CONFIANÇA a partir do texto, indique explicitamente "Não especificado/inferido na descrição". Seja honesto sobre o que PODE ser extraído.

        Descrição do Buraco: "{descricao}"

        Categorias para Extrair/Inferir da Descrição:
        - Severidade/Tamanho Estimado (Baseado na descrição): [Ex: Pequeno, Médio, Grande, Enorme, Crítico. Use termos comparativos se presentes, ex: "do tamanho de uma roda de carro".]
        - Profundidade Estimada: [Ex: Raso, Fundo, Muito Fundo. Termos como "cabe um pneu" indicam profundidade.]
        - Presença de Água/Alagamento: [Sim/Não/Não mencionado, se acumula água, vira piscina.]
        - Perigos Potenciais e Impactos Mencionados: [Liste riscos específicos citados ou implicados (ex: risco de acidente de carro/moto/bike, perigo para pedestres, causa danos a veículos - pneu furado, suspensão, roda -, dificuldade de desviar, risco de queda, perigo à noite/chuva). Seja específico.]
        - Contexto Adicional Relevante do Local/Histórico: [Problema recorrente/antigo/novo, perto de local importante (escola, hospital, comércio), em via movimentada, em curva, na esquina, na subida/descida, pouca iluminação.]
        - Sugestões de Ação/Recursos Mencionados pelo Denunciante: [Se o usuário sugere o que fazer (tapa-buraco, recapeamento, sinalizar) ou causas percebidas.]
        - Identificadores Visuais Adicionais (se descritos): [Coisas únicas próximas que ajudam a achar o buraco (poste X, árvore Y, em frente a Z).]
        - Palavras-chave Principais: [Liste 3-7 palavras-chave que capturem a essência da denúncia e o problema principal.]

        Formate a resposta de forma limpa e estruturada.
        """
        safety_settings = [ # Mantido em NONE para permitir discussão de perigos/riscos
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        response = model.generate_content(prompt, safety_settings=safety_settings)

        # Verifica se a resposta foi bloqueada por segurança ou não tem conteúdo
        if not hasattr(response, 'candidates') or not response.candidates:
             block_reason = "Desconhecido"
             if hasattr(response, 'prompt_feedback') and hasattr(response.prompt_feedback, 'block_reason'):
                 block_reason = response.prompt_feedback.block_reason.name
             print(f"❌ Análise de descrição bloqueada pelos protocolos de segurança do Gemini. Motivo: {block_reason}")
             return {"insights": f"❌ Análise de descrição bloqueada pelo filtro de segurança do Gemini. Motivo: {block_reason}"}


        # Extrai o texto da resposta
        return {"insights": response.text.strip()}

    except Exception as e:
        print(f"❌ ERRO na análise da descrição com Gemini: {e}. O Cérebro da IA deu tela azul! 😵‍💫")
        return {"insights": f"❌ Erro ao analisar a descrição com IA: {e}"}


def categorizar_urgencia_gemini(dados_denuncia: Dict[str, Any], insights_ia: Dict[str, Any], model: genai.GenerativeModel) -> Dict[str, Any]:
    """Utiliza o Gemini para sugerir uma categoria de urgência com base nos dados e insights."""
    if not model:
        return {"urgencia_ia": "🤖 Sugestão de urgência via IA indisponível (Motor Gemini offline)."}

    print("\n🧠 Calculando o Nível de Prioridade Robótica para esta denúncia...")
    try:
        descricao = dados_denuncia.get('descricao_detalhada', 'Sem descrição.')
        insights_texto = insights_ia.get('insights', 'Análise de insights não disponível.')
        localizacao_exata = dados_denuncia.get('localizacao_exata_processada', {})
        tipo_loc = localizacao_exata.get('tipo', 'Não informada')
        input_original_loc = localizacao_exata.get('input_original', 'Não informado.')


        # Formata a string de localização para dar contexto à IA
        loc_contexto = f"Localização informada: Tipo: {tipo_loc}. Detalhes originais: '{input_original_loc}'."
        if tipo_loc in ['Coordenadas Fornecidas/Extraídas Manualmente', 'Geocodificada (API)', 'Coordenadas Extraídas de Link (Manual)']:
            lat = localizacao_exata.get('latitude')
            lon = localizacao_exata.get('longitude')
            loc_contexto += f" Coordenadas: {lat}, {lon}. Link gerado: {localizacao_exata.get('google_maps_link_gerado', 'Não disponível')}."
        elif localizacao_exata.get('motivo_falha_geocodificacao_anterior'):
             loc_contexto += f" (Nota: Tentativa de Geocodificação automática falhou/não tentada: {localizacao_exata.get('motivo_falha_geocodificacao_anterior', 'Motivo desconhecido')})"


        prompt = f"""
        Com base nas informações da denúncia e nos insights extraídos pela análise anterior, sugira a MELHOR categoria de urgência para o reparo deste buraco.
        Considere a severidade/tamanho, profundidade, PERIGOS POTENCIAIS e impactos mencionados, e qualquer CONTEXTO ADICIONAL relevante (como ser recorrente, em área de alto tráfego/risco, perto de local importante).

        Escolha UMA Categoria de Urgência entre estas:
        - Urgência Baixa: Buraco pequeno, sem perigo aparente, em local de baixo tráfego. Principalmente estético ou pequeno incômodo.
        - Urgência Média: Tamanho razoável, pode causar leve incômodo ou dano menor (ex: pneu furado leve), em via secundária ou com tráfego moderado. Requer reparo em prazo razoável.
        - Urgência Alta: Buraco grande, profundo, perigo CLARO e/ou frequente (risco de acidente mais sério, dano significativo a veículo, perigo para motos/bikes/pedestres), em via movimentada ou área de risco (escola, hospital). Requer atenção RÁPIDA, possivelmente em poucos dias.
        - Urgência Imediata/Crítica: Buraco ENORME/muito profundo que causa acidentes CONSTANTES ou representa risco GRAVE e iminente a veículos ou pessoas (ex: cratera na pista principal), afeta severamente a fluidez ou acessibilidade. Requer intervenção de EMERGÊNCIA (horas/poucas horas).

        Informações Relevantes da Denúncia:
        Localização Básica: Rua {dados_denuncia.get('endereco', {}).get('rua', 'Não informada')}, Número Próximo/Referência: {dados_denuncia.get('numero_proximo', 'Não informado')}, Lado: {dados_denuncia.get('lado_rua', 'Não informado')}, Cidade: {dados_denuncia.get('endereco', {}).get('cidade_buraco', 'Não informada')}, Estado: {dados_denuncia.get('endereco', {}).get('estado_buraco', 'Não informado')}.
        {loc_contexto}
        Descrição Original do Denunciante: "{descricao}"
        Insights Extraídos pela Análise de IA:
        {insights_texto}

        Com base nestes dados, qual categoria de urgência você sugere? Forneça APENAS a categoria (ex: "Urgência Alta") e uma breve JUSTIFICATIVA (máximo 2 frases) explicando POR QUE essa categoria foi sugerida, citando elementos da descrição ou insights.

        Formato de saída (muito importante seguir este formato):
        Categoria Sugerida: [Categoria Escolhida]
        Justificativa: [Justificativa Breve]
        """
        safety_settings = [ # Mantido em NONE para permitir discussão de perigos/riscos
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        response = model.generate_content(prompt, safety_settings=safety_settings)

        if not hasattr(response, 'candidates') or not response.candidates:
             block_reason = "Desconhecido"
             if hasattr(response, 'prompt_feedback') and hasattr(response.prompt_feedback, 'block_reason'):
                 block_reason = response.prompt_feedback.block_reason.name
             print(f"❌ Sugestão de urgência bloqueada pelos protocolos de segurança do Gemini. Motivo: {block_reason}")
             return {"urgencia_ia": f"❌ Sugestão de urgência bloqueada. Motivo: {block_reason}"}


        return {"urgencia_ia": response.text.strip()}
    except Exception as e:
        print(f"❌ ERRO na sugestão de urgência com Gemini: {e}. A IA não conseguiu classificar a prioridade! 🤖💔")
        return {"urgencia_ia": f"❌ Erro ao sugerir urgência com IA: {e}"}

def sugerir_causa_e_acao_gemini(dados_denuncia: Dict[str, Any], insights_ia: Dict[str, Any], model: genai.GenerativeModel) -> Dict[str, Any]:
    """Utiliza o Gemini para sugerir possíveis causas do buraco e ações de reparo com base nos dados e insights."""
    if not model:
        return {"sugestao_acao_ia": "🤖 Sugestões de causa/ação via IA indisponíveis (Motor Gemini offline)."}

    print("\n🧠 IA está pensando... Qual pode ser a causa e a melhor ação para este buraco?")
    try:
        descricao = dados_denuncia.get('descricao_detalhada', 'Sem descrição.')
        insights_texto = insights_ia.get('insights', 'Análise de insights não disponível.')

        prompt = f"""
        Com base na descrição fornecida pelo denunciante e nos insights extraídos pela análise de IA, tente sugerir:
        1. Uma ou duas PÓSSIVEIS CAUSAS para a formação deste buraco específico (ex: chuva forte recente, desgaste do asfalto pelo tempo/tráfego, problema na drenagem subterrânea, afundamento devido a reparo anterior, obra mal feita na região). Baseie-se no contexto (se recorrente, se choveu, etc).
        2. Sugestões de TIPOS DE AÇÃO ou REPARO mais adequados ou necessários para resolver este problema (ex: simples tapa-buraco, recapeamento da seção, inspeção de drenagem, sinalização de emergência, interdição parcial da via). Baseie-se na severidade e perigos.
        Baseie suas sugestões EXCLUSIVAMENTE nas informações fornecidas na descrição e nos insights. Se a descrição não der pistas suficientes, indique "Não especificado/inferido na descrição". Seja lógico e prático.

        Informações Relevantes da Denúncia:
        Descrição Original do Buraco: "{descricao}"
        Insights Extraídos pela Análise de IA:
        {insights_texto}

        Formato de saída:
        Possíveis Causas Sugeridas: [Lista de causas sugeridas baseadas na descrição ou 'Não especificado/inferido']
        Sugestões de Ação/Reparo Sugeridas: [Lista de ações sugeridas baseadas na descrição/insights ou 'Não especificado/inferido']
        """
        safety_settings = [ # Mantido em NONE
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        response = model.generate_content(prompt, safety_settings=safety_settings)

        if not hasattr(response, 'candidates') or not response.candidates:
             block_reason = "Desconhecido"
             if hasattr(response, 'prompt_feedback') and hasattr(response.prompt_feedback, 'block_reason'):
                 block_reason = response.prompt_feedback.block_reason.name
             print(f"❌ Sugestão de causa/ação bloqueada pelos protocolos de segurança do Gemini. Motivo: {block_reason}")
             return {"sugestao_acao_ia": f"❌ Sugestão de causa/ação bloqueada. Motivo: {block_reason}"}

        return {"sugestao_acao_ia": response.text.strip()}

    except Exception as e:
        print(f"❌ ERRO na sugestão de causa/ação com Gemini: {e}. A IA não conseguiu diagnosticar o problema!")
        return {"sugestao_acao_ia": f"❌ Erro ao sugerir causa/ação com IA: {e}"}


def gerar_resumo_completo_gemini(dados_denuncia_completa: Dict[str, Any], model: genai.GenerativeModel) -> Dict[str, Any]:
    """Utiliza o Gemini para gerar um resumo narrativo inteligente da denúncia completa."""
    if not model:
        return {"resumo_ia": "🤖 Resumo inteligente via IA indisponível (Motor Gemini offline)."}

    print("\n🧠 Compilando o Relatório Final Robótico e Inteligente com IA Gemini...")
    try:
        denunciante = dados_denuncia_completa.get('denunciante', {})
        buraco = dados_denuncia_completa.get('buraco', {}) # Dados brutos do buraco
        endereco = buraco.get('endereco', {}) # Endereço básico dentro dos dados brutos
        localizacao_exata = dados_denuncia_completa.get('localizacao_exata_processada', {}) # Usa a chave processada no nível superior
        insights_ia = dados_denuncia_completa.get('insights_ia', {}).get('insights', 'Análise da descrição não disponível ou com erro.')
        urgencia_ia = dados_denuncia_completa.get('urgencia_ia', {}).get('urgencia_ia', 'Sugestão de urgência não disponível ou com erro.')
        sugestao_acao_ia = dados_denuncia_completa.get('sugestao_acao_ia', {}).get('sugestao_acao_ia', 'Sugestões de causa/ação não disponíveis ou com erro.')


        # Formatar a string de localização para o resumo
        loc_info_resumo = "Localização exata não especificada ou processada."
        tipo_loc_processada = localizacao_exata.get('tipo', 'Não informada')

        if tipo_loc_processada in ['Coordenadas Fornecidas/Extraídas Manualmente', 'Geocodificada (API)', 'Coordenadas Extraídas de Link (Manual)']:
             lat = localizacao_exata.get('latitude')
             lon = localizacao_exata.get('longitude')
             link_gerado = localizacao_exata.get('google_maps_link_gerado', 'Não disponível')
             loc_info_resumo = f"Localização: Coordenadas {lat}, {lon} (Obtida via: {tipo_loc_processada.replace(' (API)', ' API').replace('Manual', 'Manual').replace('Fornecidas/Extraídas', 'Manual')}). Link Google Maps: {link_gerado}."
             if localizacao_exata.get('input_original'):
                 loc_info_resumo += f" (Input original: '{localizacao_exata.get('input_original')}')"

        elif tipo_loc_processada == 'Descrição Manual Detalhada':
             loc_info_resumo = f"Localização via descrição manual detalhada: '{localizacao_exata.get('descricao_manual', 'Não informada')}'. (Input original: '{localizacao_exata.get('input_original', 'Não informado')}')"

        elif localizacao_exata.get('input_original') and tipo_loc_processada == 'Não informada': # Se não identificamos o tipo mas temos o input original
             loc_info_resumo = f"Localização informada (tipo não detectado): '{localizacao_exata.get('input_original')}'."

        if localizacao_exata.get('motivo_falha_geocodificacao_anterior'):
             loc_info_resumo += f" (Nota: Geocodificação automática falhou/não tentada: {localizacao_exata.get('motivo_falha_geocodificacao_anterior')})"


        prompt = f"""
        Gere um resumo narrativo conciso (máximo 8-10 frases) para a seguinte denúncia de buraco no aplicativo Krateras.
        Este resumo deve ser formal, objetivo e útil para equipes de manutenção ou gestão pública.
        Combine os dados estruturados, a localização exata processada e os insights das análises de IA.

        Inclua:
        - Quem denunciou (Nome, Cidade de Residência).
        - Onde está o buraco (Rua, Número Próximo/Referência, Bairro, Cidade do Buraco, Estado do Buraco, CEP se disponível).
        - A localização EXATA (mencione como foi obtida - Geocodificada, Coordenadas Manual, Descrição Manual - e inclua os dados - Coordenadas/Link Gerado ou Descrição Manual).
        - O lado da rua.
        - Os principais pontos da Análise Detalhada de IA sobre o buraco (Severidade/Tamanho, Perigos Potenciais, Contexto Relevante).
        - A SUGESTÃO de Categoria de Urgência pela IA e sua Justificativa.
        - As SUGESTÕES de POSSÍVEIS CAUSAS e TIPOS DE AÇÃO/REPARO sugeridas pela IA (se disponíveis).

        Dados da Denúncia:
        Denunciante: {denunciante.get('nome', 'Não informado')}, de {denunciante.get('cidade_residencia', 'Não informada')}.
        Endereço do Buraco: Rua {endereco.get('rua', 'Não informada')}, Nº Próximo: {buraco.get('numero_proximo', 'Não informado')}. Bairro: {endereco.get('bairro', 'Não informado')}. Cidade: {endereco.get('cidade_buraco', 'Não informada')}, Estado: {endereco.get('estado_buraco', 'Não informado')}. CEP: {buraco.get('cep_informado', 'Não informado')}.
        Lado da Rua: {buraco.get('lado_rua', 'Não informado')}.
        Localização Exata Coletada: {loc_info_resumo}
        Descrição Original: "{buraco.get('descricao_detalhada', 'Não fornecida.')}"

        Insights da Análise Detalhada de IA:
        {insights_ia}

        Sugestão de Urgência pela IA:
        {urgencia_ia}

        Sugestões de Causa e Ação pela IA:
        {sugestao_acao_ia}


        Gere o resumo em português. Comece com "Relatório Krateras: Denúncia de buraco..." ou algo similar. Use linguagem clara e direta.
        """
        safety_settings = [ # Mantido em NONE para permitir discussão de perigos/riscos
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        response = model.generate_content(prompt, safety_settings=safety_settings)

        if not hasattr(response, 'candidates') or not response.candidates:
             block_reason = "Desconhecido"
             if hasattr(response, 'prompt_feedback') and hasattr(response.prompt_feedback, 'block_reason'):
                 block_reason = response.prompt_feedback.block_reason.name
             print(f"❌ Geração de resumo bloqueada pelos protocolos de segurança do Gemini. Motivo: {block_reason}")
             return {"resumo_ia": f"❌ Geração de resumo bloqueada. Motivo: {block_reason}"}

        return {"resumo_ia": response.text.strip()}
    except Exception as e:
        print(f"❌ ERRO na geração do resumo com Gemini: {e}. O compilador de relatório teve um bug!")
        return {"resumo_ia": f"❌ Erro ao gerar resumo completo com IA: {e}"}


# --- 📊 Funções de Saída Final ---
# Apresenta o relatório completo com dados coletados e análises de IA.

def exibir_resumo_denuncia(dados_completos: Dict[str, Any]):
    """Exibe um resumo formatado da denúncia completa, incluindo todas as análises de IA."""
    denunciante = dados_completos.get('denunciante', {})
    buraco = dados_completos.get('buraco', {})
    endereco = buraco.get('endereco', {})
    localizacao_exata = dados_completos.get('localizacao_exata_processada', {}) # Usa a chave processada
    insights_ia = dados_completos.get('insights_ia', {})
    urgencia_ia = dados_completos.get('urgencia_ia', {})
    sugestao_acao_ia = dados_completos.get('sugestao_acao_ia', {}) # Nova chave
    resumo_ia = dados_completos.get('resumo_ia', {})

    print("\n" + "="*60)
    print("         📊 RELATÓRIO FINAL DA DENÚNCIA KRATERAS 📊")
    print("="*60)

    print("\n--- 👤 Dados do Denunciante ---")
    print(f"Nome: {denunciante.get('nome', 'Não informado')}")
    print(f"Idade: {denunciante.get('idade', 'Não informado')}")
    print(f"Cidade de Residência: {denunciante.get('cidade_residencia', 'Não informada')}") # Chave atualizada

    print("\n--- 🚧 Dados do Buraco ---")
    print(f"Rua: {endereco.get('rua', 'Não informada')}")
    if buraco.get('numero_proximo'):
        print(f"Referência/Número Próximo: {buraco.get('numero_proximo')}")
    if endereco.get('bairro'):
        print(f"Bairro: {endereco.get('bairro')}")
    if endereco.get('cidade_buraco'): # Chave atualizada
         print(f"Cidade do Buraco: {endereco.get('cidade_buraco')}")
    if endereco.get('estado_buraco'): # Chave atualizada
        print(f"Estado do Buraco: {endereco.get('estado_buraco')}")
    if buraco.get('cep_informado'):
        print(f"CEP Informado: {buraco.get('cep_informado')}")

    print(f"Lado da Rua: {buraco.get('lado_rua', 'Não informado')}")


    print("\n--- 📍 Localização Exata ---")
    tipo_loc = localizacao_exata.get('tipo', 'Não informada')
    print(f"Tipo de Coleta: {tipo_loc}")

    if tipo_loc in ['Coordenadas Fornecidas/Extraídas Manualmente', 'Geocodificada (API)', 'Coordenadas Extraídas de Link (Manual)']:
        print(f"Coordenadas: {localizacao_exata.get('latitude')}, {localizacao_exata.get('longitude')}")
        if localizacao_exata.get('google_maps_link_gerado'):
            print(f"Link Google Maps Gerado: {localizacao_exata.get('google_maps_link_gerado')}")
        if localizacao_exata.get('endereco_formatado_api'):
             print(f"Endereço Formatado (API): {localizacao_exata.get('endereco_formatado_api')}")
        print(f"(Input Original: '{localizacao_exata.get('input_original', 'Não informado')}')")

    elif tipo_loc == 'Descrição Manual Detalhada':
        print(f"Descrição Manual: {localizacao_exata.get('descricao_manual', 'Não informada')}")
        print(f"(Input Original: '{localizacao_exata.get('input_original', 'Não informado')}')")

    else:
        print("Localização exata não coletada de forma estruturada.")

    # Inclui motivo da falha na geocodificação se aplicável
    if localizacao_exata.get('motivo_falha_geocodificacao_anterior'):
         print(f"ℹ️ Nota: Não foi possível obter a localização exata via Geocodificação automática. Motivo: {localizacao_exata.get('motivo_falha_geocodificacao_anterior')}")


    print("\n--- 📝 Descrição Original do Denunciante ---")
    print(buraco.get('descricao_detalhada', 'Nenhuma descrição fornecida.'))


    print("\n--- 🧠 Análise Detalhada da Descrição (IA Gemini) ---")
    print(insights_ia.get('insights', 'Análise não realizada ou com erro.'))


    print("\n--- 🚦 Sugestão de Urgência (IA Gemini) ---")
    print(urgencia_ia.get('urgencia_ia', 'Sugestão de urgência não gerada ou com erro.'))


    print("\n--- 🛠️ Sugestões de Causa e Ação (IA Gemini) ---")
    print(sugestao_acao_ia.get('sugestao_acao_ia', 'Sugestões não geradas ou com erro.')) # Nova exibição


    print("\n--- 📜 Resumo Narrativo Inteligente (IA Gemini) ---")
    print(resumo_ia.get('resumo_ia', 'Resumo não gerado ou com erro.'))


    print("\n" + "="*60)
    print("        ✅ MISSÃO KRATERAS CONCLUÍDA! RELATÓRIO GERADO. ✅")
    print("="*60)
    print("\nEsperamos que este relatório ajude a consertar o buraco!")


# --- 🚀 Função Principal: O Orquestrador ---

def main():
    # 1. Instalar dependências (executado no início do script)

    # 2. Obter chaves de API (agora tenta Segredos para ambas)
    gemini_api_key, geocoding_api_key = get_api_keys()

    # 3. Inicializar Google Gemini
    gemini_model = init_gemini(gemini_api_key)

    # 4. Coletar dados do denunciante
    denunciante = coletar_dados_denunciante()

    # 5. Coletar dados do buraco (passa a chave de geocodificação opcional)
    # Esta função retorna todos os dados brutos do buraco MAIS a localização processada.
    buraco_data = coletar_dados_buraco(geocoding_api_key)

    # Estrutura completa da denúncia. Move a localização processada para o nível superior.
    denuncia_completa: Dict[str, Any] = {
        "denunciante": denunciante,
        "buraco": {k: v for k, v in buraco_data.items() if k != 'localizacao_exata_processada'}, # Copia tudo exceto a localização processada
        "localizacao_exata_processada": buraco_data.get('localizacao_exata_processada') # Adiciona a localização processada no nível superior
    }


    # 6. Rodar análises de IA (se o modelo Gemini foi inicializado)
    if gemini_model:
        # Análise da descrição
        insights = analisar_descricao_gemini(buraco_data.get('descricao_detalhada', ''), gemini_model)
        denuncia_completa['insights_ia'] = insights

        # Sugestão de urgência (usa dados do buraco e insights)
        # Passa buraco_data (que contém os dados básicos do buraco) E os insights_ia para dar contexto à IA
        urgencia = categorizar_urgencia_gemini(buraco_data, denuncia_completa['insights_ia'], gemini_model)
        denuncia_completa['urgencia_ia'] = urgencia

        # Sugestão de causa e ação (usa dados do buraco e insights)
        # Passa buraco_data E os insights_ia para dar contexto à IA
        sugestao_acao = sugerir_causa_e_acao_gemini(buraco_data, denuncia_completa['insights_ia'], gemini_model)
        denuncia_completa['sugestao_acao_ia'] = sugestao_acao


        # Geração do resumo (usa todos os dados coletados e resultados da IA)
        # Passa a denuncia_completa completa que agora tem todos os dados + resultados das outras IAs
        resumo_ia = gerar_resumo_completo_gemini(denuncia_completa, gemini_model)
        denuncia_completa['resumo_ia'] = resumo_ia
    else:
         print("\n⚠️ Funcionalidades de Análise e Resumo da IA não executadas (Gemini não configurado).")
         denuncia_completa['insights_ia'] = {"insights": "Análise de descrição via IA indisponível."}
         denuncia_completa['urgencia_ia'] = {"urgencia_ia": "Sugestão de urgência via IA indisponível."}
         denuncia_completa['sugestao_acao_ia'] = {"sugestao_acao_ia": "Sugestões de causa/ação via IA indisponíveis."}
         denuncia_completa['resumo_ia'] = {"resumo_ia": "Resumo completo via IA indisponível."}


    # 7. Exibir o relatório completo
    exibir_resumo_denuncia(denuncia_completa)

    # 8. Ponto para salvar os dados (opcional)
    # Os dados completos da denúncia estão no dicionário `denuncia_completa`.
    # Você pode salvá-los em um arquivo JSON, Google Sheets, etc.
    # print("\n🔌 Dados brutos da denúncia (formato JSON):")
    # print(json.dumps(denuncia_completa, indent=4, ensure_ascii=False))
    # try:
    #     with open("denuncia_krateras.json", "w", encoding='utf-8') as f:
    #         json.dump(denuncia_completa, f, indent=4, ensure_ascii=False)
    #     print("\n💾 Dados da denúncia salvos em 'denuncia_krateras.json' no ambiente do Colab.")
    # except Exception as e:
    #     print(f"\n❌ ERRO ao tentar salvar os dados em arquivo: {e}")


if __name__ == "__main__":
    main()

✅ Módulo google.colab.userdata disponível. Ambiente Colab detectado. Tentarei usar Segredos.
✨ Krateras v3.2 Inicializando seus sistemas...
🔧 Realizando varredura de componentes essenciais e instalando ferramentas...
🛰️ Ferramenta 'google-generativeai' não encontrada. Baixando e instalando do meu repositório central...
'google-generativeai' instalado com sucesso. Montagem completa!

--- 🔑 Inserindo Chaves de Acesso Seguras (Energia Secreta para os Robôs!) ---
Vou tentar carregar suas chaves dos Segredos do Colab para máxima segurança.
Certifique-se de ter os segredos 'GOOGLE_API_KEY' (para Gemini) e 'geocoding_api_key' (para Geocoding) configurados.

1. Chave Google AI Studio/Vertex AI (Gemini):
💾 Tentando obter a chave 'GOOGLE_API_KEY' dos Segredos do Colab...
✅ Chave 'GOOGLE_API_KEY' encontrada nos Segredos do Colab!

2. Chave Google Cloud Platform (Geocoding API):
💾 Tentando obter a chave 'geocoding_api_key' dos Segredos do Colab...
✅ Chave 'geocoding_api_key' encontrada nos Segredo